In [19]:
import numpy as np
import pandas as pd

In [20]:
df = pd.read_parquet('yellow_tripdata_2023-01.parquet', engine='pyarrow')


In [53]:
df.head(n=10).T

,0,1,2,3,4,5,6,7,8,9
VendorID,2,2,2,1,2,2,2,2,2,2
tpep_pickup_datetime,2023-01-01 00:32:10,2023-01-01 00:55:08,2023-01-01 00:25:04,2023-01-01 00:03:48,2023-01-01 00:10:29,2023-01-01 00:50:34,2023-01-01 00:09:22,2023-01-01 00:27:12,2023-01-01 00:21:44,2023-01-01 00:39:42
tpep_dropoff_datetime,2023-01-01 00:40:36,2023-01-01 01:01:27,2023-01-01 00:37:49,2023-01-01 00:13:25,2023-01-01 00:21:19,2023-01-01 01:02:52,2023-01-01 00:19:49,2023-01-01 00:49:56,2023-01-01 00:36:40,2023-01-01 00:50:36
passenger_count,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
trip_distance,0.97,1.1,2.51,1.9,1.43,1.84,1.66,11.7,2.95,3.01
RatecodeID,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
store_and_fwd_flag,N,N,N,N,N,N,N,N,N,N
PULocationID,161,43,48,138,107,161,239,142,164,141
DOLocationID,141,237,238,7,79,137,143,200,236,107
payment_type,2,1,1,1,1,1,1,1,1,2


In [23]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [27]:
df.duration.describe().round(2)

count    3066766.00
mean          15.67
std           42.59
min          -29.20
25%            7.12
50%           11.52
75%           18.30
max        10029.18
Name: duration, dtype: float64

In [29]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
df.duration.describe().round(2)

count    3009173.00
mean          14.20
std            9.94
min            1.00
25%            7.22
50%           11.55
75%           18.18
max           60.00
Name: duration, dtype: float64

In [31]:
3009173.00/3066766.00*100

98.1220282212598

In [58]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

In [41]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [59]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)



7.649261952121727

In [45]:
df_val = pd.read_parquet('yellow_tripdata_2023-02.parquet', engine='pyarrow')


In [46]:
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

In [48]:
df_val.duration.describe().round(2)

count    2913955.00
mean          16.02
std           42.84
min          -43.62
25%            7.25
50%           11.80
75%           18.77
max         7053.62
Name: duration, dtype: float64

In [61]:
df_val[categorical] = df_val[categorical].astype(str)
train_dicts_val = df_val[categorical].to_dict(orient='records')



dv = DictVectorizer()
X_val = dv.fit_transform(train_dicts_val)

target = 'duration'
y_val = df_val[target].values
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)


ValueError: X has 519 features, but LinearRegression is expecting 515 features as input.

In [54]:
y_pred = lr.predict(X_train)

In [57]:
?X_train

Type:        csr_matrix
String form:
(0, 43)	1.0
           (0, 325)	1.0
           (1, 148)	1.0
           (1, 456)	1.0
           (2, 149)	1.0
           (2, 461)	1.0
           (3, 22 <...> 9170, 273)	1.0
           (3009171, 237)	1.0
           (3009171, 400)	1.0
           (3009172, 45)	1.0
           (3009172, 435)	1.0
File:        c:\users\sebastiansy\appdata\local\anaconda3\lib\site-packages\scipy\sparse\_csr.py
Docstring:  
Compressed Sparse Row matrix

This can be instantiated in several ways:
    csr_array(D)
        with a dense matrix or rank-2 ndarray D

    csr_array(S)
        with another sparse matrix S (equivalent to S.tocsr())

    csr_array((M, N), [dtype])
        to construct an empty matrix with shape (M, N)
        dtype is optional, defaulting to dtype='d'.

    csr_array((data, (row_ind, col_ind)), [shape=(M, N)])
        where ``data``, ``row_ind`` and ``col_ind`` satisfy the
        relationship ``a[row_ind[k], col_ind[k]] = data[k]``.

    csr_array((data,